In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Prompt for file name, minimum time, and maximum time
file_name = input("Enter the file name: ")
min_time = float(input("Enter the minimum time: "))
max_time = float(input("Enter the maximum time: "))
n=max_time
data = pd.read_csv(file_name)
time = data.iloc[:, 0]
data_values = data.iloc[:, 1]

# Filtering data within the specified time range
mask = (time >= min_time) & (time <= max_time)
time_filtered = time[mask]
data_values_filtered = data_values[mask]

# Calculating the mean value for each unique time
unique_times = np.unique(time_filtered)
data_mean = []
for t in unique_times:
    indices = np.where(time_filtered == t)[0]
    if len(indices) > 0:
        mean_value = np.mean(data_values_filtered.iloc[indices])  # Use .iloc indexing for DataFrame
        data_mean.append(mean_value)

x = unique_times
y = np.array(data_mean)

# Model function
def model_func(x, m, a, b):
    return a * np.exp(-m * x) + b * np.exp(-n* m * x)

# Fit with different windows
fit_windows = np.linspace(0, len(x) - 10, 50, dtype=int)

for start in fit_windows:
    end = start + 10

    # Fit
    params, _ = curve_fit(model_func, x[start:end], y[start:end], maxfev=10000)

    # Model predictions
    model_predictions = model_func(x, *params)

    # Calculating the residuals
    residuals = y - model_predictions
    ss_residuals = np.sum(residuals**2)
    ss_total = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_residuals / ss_total)

    # Calculating the chi-square value
    chi_squared = np.sum(residuals**2) / y

    m_value, _, _ = params
    print(f'Fit Window: {start}-{end}')
    print('------------------------')
    print("m:", m_value)
    print("Chi-Square:", chi_squared)
    print()
